98106434 <span style='font-size:larger;'>→</span> Mohammad Khodadadi Aski
<br>
98105919 <span style='font-size:larger;'>→</span> Reza Erfan Arani
<br>
98101566 <span style='font-size:larger;'>→</span> Mohammadreza Daviran
<br>
<br>
<a href='https://github.com/rerfan24/MIR-HW1' style='text-align:center;'>github repo</a> 
<br>


We use the BeautifulSoup in order to extraxt data from the request we send to the website .
.strip method removes any unexpected and useless whitespace
In order to run the code , you should run the bellow cell to installl required libraries.

In [ ]:
!pip install hazm
!pip install selenium
!pip install bs4

In the code bellow we do the following tasks:
<br>
Normalization :
<br>
In normalization we delete the spaces which dont have any meaning .
<br>
Tokenized_sent_input <span style='font-size:larger;'>→</span> We use the sent_tokenize method of hazm library in order to tokenize the sentences of our input.
<br>
Tokenized_word_input <span style='font-size:larger;'>→</span> We use the word_tokenize method of hazm library to tokenize our input word by word.
<br>
Then we stem our normlized input in order to send the words to their root word, but we dont use it in this project because of our tasks which we can afford loosing any data.
<br>
We have the same thing with the lemmatizer.

In [212]:
from __future__ import unicode_literals
from hazm import *

def run (input:str):
    normalizer = Normalizer()
    normilized_input=normalizer.normalize(input)
    tokenized_sent_input = sent_tokenize(input)
    tokenized_word_input = word_tokenize(input)
    stemmer = Stemmer()
    stemmed_input = stemmer.stem(normilized_input)
    # we did this , but we shouldnt use this data cause some data are lost during this pre-proccessing phase
    print(f'{tokenized_sent_input}\n{tokenized_word_input}')
    indices = [i for i, x in enumerate(tokenized_word_input) if x.startswith('دستور')]
    print(indices)
    lemmatizer = Lemmatizer()
    lemmed_input = lemmatizer.lemmatize(normilized_input)
    # we also have the same thing about lemmatizing (same as stemming)

In the following function, we extracted recipe from the data using the index given.

In [229]:
def find_recipe(input_recipe, index):
    recipe = input_recipe[index:]
    newline_index = recipe.index(chr(10)) 
    recipe = recipe[newline_index + 1:]
    return recipe, [index + newline_index + 1, len(input_recipe)]

In the following code block, we find the spans of different ingredients, in order to do so we search our data string and find the ingredient given in the text and we return the starting index and ending index of each ingredient in the given text.

In [239]:
def findSpanIngredient(ingut: str, ingredients):
    span = []
    last = 0
    for ing in ingredients:
        try: 
            first = ingut.index(ing)
        except:
            first = last + 1
        while (last <= first):
            last = ingut.index(chr(10),last+1)
        if (not ingut[first:last].startswith('دستور شمار')):
            span.append([first,last])
    return span

The following code block is written to find ingredients in our string. <br>
We split our text by 3 splitors, each showing a specific data part in our input string. <br>
We use our spliter array which contains the amounts in both numbers and uncountable quantities to split data one more time so that we could extract amount for each ingredient seprately. from start of the line to our splitter index will be our ingredient and from our ingredient index to the end of the line will be our ingredient amount.

In [252]:
def find_ingredients(ingut : str):
    start_time = time.time()
    
    ingredients = []
    quantity = []
    span_ingredients = []
    if "مواد" in ingut.split("\n")[0]:
        starting_index = ingut.index("مواد")
    else:
        starting_index = 0
    prepare_spliter = ["طرز تهیه", "روش تهیه", "طرزتهیه"]
    ending_index = 0
    for element in prepare_spliter:
            try:
                ending_index = ingut.index(element)
                break
            except:
                continue
    
    ing_data = ingut[starting_index:ending_index]
    if "مواد" in ingut.split("\n")[0]:
        ing_data= ing_data.split("\n",1)[1]
    if "مواد" in ingut.split("\n")[1]:
        ing_data= ing_data.split("\n",1)[1]
        ing_data= ing_data.split("\n",1)[1]
        
    spliter = ['0', '۰','1', '۱', '2', '۲', '3', '۳', '4', '۴', '5', '۵', '6',
               '۶', '7', '۷', '8', '۸', '9', '۹', 'یک ', 'یک‌',
               'دو ', 'دو‌', 'سه ', 'سه‌','نیم', 'نیم‌', 'به مقدار لازم', 'به میزان لازم',
               'نصف ', 'نصف‌','مقداری', 'به میزان کافی', 'کمتر از', 'دوقاشق']
    
    while (True):    
        spliterIndex = []
        for i in spliter:
            try:
                spliterIndex.append(ing_data.index(i))
            except:
                spliterIndex.append(9223372036854775807)
        endline = 0
        try:
            endline = ing_data.index(chr(10)) 
        except:
            pass
        middle = min(spliterIndex)
        
        if (middle > endline):
            middle = endline
        
        ing = ing_data[0:middle].strip()
        quant = ing_data[middle:endline].strip()
        
        ing = ing.replace('=', '').strip()
        ing = ing.replace(':', '').strip()
        
        ingredients.append(ing)
        quantity.append(quant)
        ing_data = ing_data[endline+1:]
        if (len(ing_data) == 0):
            break
            
    recipe, span_recipe = find_recipe(ingut ,ending_index)
    span_ingredients = findSpanIngredient(ingut, ingredients)
    
    end_time = time.time()
    
    return (ingredients, quantity, recipe, span_ingredients, span_recipe, end_time - start_time)
   

This function is written for the foods with multiple recipees and returns an array which contains each recipee starting index.

In [253]:
def findRecipeIndex(s: str, ch: str):
    indexes = []
    first = 0
    while (True):
        try:
            first = s.index(ch, first)
            indexes.append(first)
            first += 1
        except:
            break
    return indexes

In the following code block we do the following steps which forms our Main:
1. We read our data from the file.
2. Then we find how many way we have to cook the selected food.
3. Then we iterate each way and we call our functions to find ingredients, quantity of them, recipe, span_ingredients and time.
4. We build a dictionary as requested on our homework and we add our data to it.

In [254]:
import time

lines = ''
with open('input.txt', 'r',  encoding='utf-8') as f:
    lines = f.read()
all_dicts = []
indexes = findRecipeIndex(lines, 'دستور شمار')
if len(indexes) == 0:
    indexes.append(0)
indexes.append(len(lines))
for i in range(len(indexes)-1):
    first = indexes[i]
    if indexes[i] != 0:
        first = lines.find(chr(10),first) + 1
    last = indexes[i+1]
    ingredients, quantity, recipe, span_ingredients, span_recipe, time_passed = find_ingredients(lines[first:last])
    final_dict = {}
    final_dict.update({'Ingredients': ingredients})
    final_dict.update({'quantity': quantity})
    final_dict.update({'recipie': recipe})
    final_dict.update({'span_ingredients': span_ingredients})
    final_dict.update({'span_recipie': span_recipe})
    final_dict.update({'time': time_passed})
    all_dicts.append(final_dict)
all_dicts

[{'Ingredients': ['',
   'گوشت گوسفندی با استخوان',
   'دنبه',
   'لپه',
   'پیاز متوسط',
   'مغز لیمو عمانی',
   'آب گوجه\u200cفرنگی',
   'رب گوجه\u200cفرنگی',
   'سیب\u200cزمینی',
   'نمک و فلفل'],
  'quantity': ['',
   '۱ کیلو',
   '۳۰۰ گرم',
   'یک فنجان سرخالی',
   'یک عدد',
   'یک قاشق سوپخوری',
   'یک فنجان',
   'یک قاشق سوپخوری',
   '۵ عدد',
   'به مقدار لازم'],
  'recipie': '\nگوشت و دنبه را پس از شستن ۱۰ قسمت کرده و آن را سرخ کنید تا طلایی شود. پیاز را در روغن سرخ نمایید و لپه خیس\u200cداده\u200cشده را به پیاز اضافه کرده و لپه را هم کمی تفت دهید.\n\nگوشت سرخ\u200cشده را با ۴ لیوان آب، زردچوبه، لپه و پیاز داغ با شعله ملایم بپزید. سپس مغز لیمو عمانی را به آبگوشت اضافه کنید. در اواخر طبخ، ۵ تا ۶ عدد سیب\u200cزمینی را پوست گرفته و با کمی نمک، فلفل و زردچوبه به غذا اضافه می\u200cکنیم. بعد هم آب گوجه\u200cفرنگی و رب گوجه\u200cفرنگی را به آن اضافه کنید. نان سنگک، پیاز، دوغ، سبزی خوردن در کنارش فراموش نشود.',
  'span_ingredients': [[41, 73],
   [74, 88],
   [89, 110],
   [111, 130],
